In [1]:
import base
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
pi = np.array([0.1, 0.4, 0.4, 0.1])

In [3]:
def calculer_q(env, v, gamma):
    return np.sum(env.p() * 
            (env.r() + gamma*v[np.newaxis, np.newaxis, :]), axis=2)

In [4]:
def ipe(env, pi, gamma, epsilon):
    ns = env.get_nb_states()
    v = np.zeros((ns))
    q = calculer_q(env, v, gamma)
    nv = np.sum(pi * q, axis=1)
    
    delta = np.sum(np.abs(nv - v))
    while delta > epsilon:
        v = nv
        q = calculer_q(env, v, gamma)
        nv = np.sum(pi * q, axis=1)
        delta = np.sum(np.abs(nv - v))
        
    return nv 

In [5]:
def monte_carlo_predict(env, pi, gamma, nbEpisode, epsilon):
    na = env.get_nb_actions()
    ns = env.get_nb_states()
    q = np.zeros((env.get_nb_states()))
    n = np.zeros((env.get_nb_states()))

    #définition de la politique par défaut
    if pi is None:
        pi = np.ones((ns, na))/na

    V2 = ipe(env, pi, gamma, epsilon)
    V = np.zeros((ns))
    error = []

    for k in range(1, nbEpisode+1):
        # Faire un episode
        s = env.reset()
        episode = []
        while not env.is_final(s):
            a = np.random.choice(na, p=pi) 
            ns, r, _, _ = env.step(a)
            episode.append([s, a, ns, r])
            s = ns
            
        # Mettre à jour Q
        retour = 0
        for s, a, ns, r in reversed(episode):
            retour = r + gamma * retour
            n[s] += 1
            q[s] += retour
            V[s] = q[s]/n[s]
        
        current_error = np.linalg.norm(V2-V)
        error.append(current_error)
        
    #return q/n
    return error

In [6]:
error = monte_carlo_predict(env, pi, 0.9, 100, 0.01)

plt.title("Erreur entre V(ipe) et V à chaque épisode")
plt.plot(error, color="red")

plt.show()

NameError: name 'env' is not defined